*Define Problem:*

There are multiple vehicles with different capacities and locations that need to make deliveries to various destinations. The goal is to find the most efficient routes for each vehicle that minimize the total distance traveled and maximize the number of deliveries made.

Suppose there are 5 vehicles and 10 delivery locations. The capacities of the vehicles are:

*  Vehicle 1: 3
*  Vehicle 2: 4
*  Vehicle 3: 2
*  Vehicle 4: 5
*  Vehicle 5: 3

The coordinates of the delivery locations are:

*  Location 1: (2, 5)
*  Location 2: (8, 3)
*  Location 3: (9, 9)
*  Location 4: (4, 7)
*  Location 5: (1, 2)
*  Location 6: (6, 4)
*  Location 7: (3, 8)
*  Location 8: (7, 1)
*  Location 9: (5, 6)
*  Location 10: (10, 10)

Each delivery location has a demand value indicating the number of deliveries that need to be made there:

*  Location 1: 2
*  Location 2: 3
*  Location 3: 1
*  Location 4: 2
*  Location 5: 1
*  Location 6: 3
*  Location 7: 1
*  Location 8: 2
*  Location 9: 1
*  Location 10: 2

The goal is to find the most efficient routes for each vehicle that minimize the total distance traveled and maximize the number of deliveries made.

*Import Library*

In [211]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

*Function*

In [212]:
# define constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.01
ELITE_SIZE = 2
GENERATIONS = 100

In [213]:
vehicle_0 = [[0],[0],[3]] #X,Y and capacity
vehicle_1 = [[0],[0],[4]]
vehicle_2 = [[0],[0],[2]]
vehicle_3 = [[0],[0],[5]]
vehicle_4 = [[0],[0],[3]]

Location_0 = [[2],[5],[2]] #X,Y and demand
Location_1 = [[8],[3],[3]]
Location_2 = [[9],[9],[1]]
Location_3 = [[4],[7],[2]]
Location_4 = [[1],[2],[1]]
Location_5 = [[6],[4],[3]]
Location_6 = [[3],[8],[1]]
Location_7 = [[7],[1],[2]]
Location_8 = [[5],[6],[1]]
Location_9 = [[10],[10],[2]]

In [214]:
location_list = [Location_0, Location_1, Location_2, Location_3, Location_4, 
             Location_5, Location_6, Location_7, Location_8, Location_9]
vehicle_list = [vehicle_0, vehicle_1, vehicle_2, vehicle_3, vehicle_4]

In [215]:
def calculatedistance(vehicle, location):
  x1, y1 = vehicle[0][0], vehicle[1][0]
  x2, y2 = location[0][0], location[1][0]
  distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
  # print(f'Distance:{distance}')
  return distance

In [216]:
calculatedistance(vehicle_4,Location_3)

8.06225774829855

In [217]:
def create_individual():
    # Creates a new individual (i.e. a list of deliveries for each vehicle)
    individual = []
    for i in range(len(vehicle_list)):
        deliveries = []
        for j in range(len(location_list)):
            if location_list[j][2][0] <= vehicle_list[i][2][0]:
                deliveries.append(j)
        individual.append(deliveries)
    return individual

In [218]:
def calculate_fitness(individual):
    # Calculates the fitness of an individual by summing the distance traveled by each vehicle
    total_distance = 0
    total_deliveries = 0
    for i in range(len(individual)):
        vehicle = vehicle_list[i]
        deliveries = individual[i]
        if len(deliveries) == 0:
            continue
        # Calculate the distance from the vehicle's starting location to the first delivery location
        total_distance += calculatedistance(vehicle, location_list[deliveries[0]])
        # Calculate the distance between subsequent delivery locations
        for j in range(len(deliveries) - 1):
            total_distance += calculatedistance(location_list[deliveries[j]], location_list[deliveries[j+1]])
        # Calculate the distance from the last delivery location to the vehicle's starting location
        total_distance += calculatedistance(location_list[deliveries[-1]], vehicle)
        # Increment the number of deliveries made by the vehicle
        total_deliveries += len(deliveries)
    # Calculate the fitness as a combination of the total distance traveled (minimize) and the number of deliveries made (maximize)
    fitness = 1 / (total_distance + total_deliveries)
    return fitness

In [219]:
def crossover(parent1, parent2):
    # Performs crossover between two parents by selecting a random delivery from each parent for each vehicle
    child = []
    for i in range(len(parent1)):
        deliveries = []
        for j in range(len(parent1[i])):
            if random.random() < 0.5:
                deliveries.append(parent1[i][j])
            else:
                deliveries.append(parent2[i][j])
        child.append(deliveries)
    return child

In [220]:
def mutate(individual, MUTATION_RATE=0.1):
    # Performs mutation on an individual by randomly selecting a delivery for each vehicle and adding or removing it
    for i in range(len(individual)):
        if random.random() < MUTATION_RATE:
            deliveries = individual[i]
            if len(deliveries) == 0:
                deliveries.append(random.randint(0, len(location_list) - 1))
            elif len(deliveries) == 1:
                if random.random() < 0.5:
                    deliveries.append(random.randint(0, len(location_list) - 1))
                else:
                    deliveries = []
            else:
                if random.random() < 0.5:
                    deliveries.pop(random.randint(0, len(deliveries) - 1))
                else:
                    deliveries.insert(random.randint(0, len(deliveries) - 1), random.randint(0, len(location_list) - 1))
            individual[i] = deliveries
    return individual

In [221]:
def tournament_selection(population, fitnesses):
    # Set the tournament size to be 2
    tournament_size = 2
    # Initialize the selected individuals list
    selected_individuals = []
    # Perform tournament selection until we have selected enough individuals
    while len(selected_individuals) < len(population):
        # Randomly select tournament_size individuals from the population
        tournament = random.sample(range(len(population)), tournament_size)
        # Find the index of the individual with the highest fitness score in the tournament
        winner_index = max(tournament, key=lambda k: fitnesses[k])
        # Add the winner to the list of selected individuals
        selected_individuals.append(population[winner_index])
    return selected_individuals

In [222]:
# Initialize the population
population = [create_individual() for i in range(POPULATION_SIZE)]

In [223]:
# Main loop
for generation in range(GENERATIONS):
    print(f'=== Gen{generation} === ')
    # Calculate the fitness of each individual
    fitnesses = [calculate_fitness(individual) for individual in population]
    # Select the elites (i.e. the top 2 individuals)
    elites = sorted(range(len(fitnesses)), key=lambda k: fitnesses[k], reverse=True)[:2]
    # Perform selection and crossover to create the next generation
    # Initialize the next generation with the elites
    next_generation = [population[i] for i in elites]

    # Create the rest of the population through selection and crossover
    while len(next_generation) < POPULATION_SIZE:
        # Select two parents using tournament selection
        parent1 = tournament_selection(population, fitnesses)
        parent2 = tournament_selection(population, fitnesses)

        # Perform crossover to create two children
        children = crossover(parent1, parent2)
        child1 = children[0]
        child2 = children[1]

        # Mutate the children
        child1 = mutate(child1, MUTATION_RATE)
        child2 = mutate(child2, MUTATION_RATE)

        # Add the children to the next generation
        next_generation.append(child1)
        if len(next_generation) < POPULATION_SIZE:
            next_generation.append(child2)

    # Update the population
    population = next_generation

    # Print the best individual of the current generation
    best_individual_index = max(range(len(fitnesses)), key=lambda k: fitnesses[k])
    best_individual = population[best_individual_index]
    best_fitness = fitnesses[best_individual_index]
    print(f"Generation {generation}: Best individual = {best_individual}, Best fitness = {best_fitness}")

=== Gen0 === 
Generation 0: Best individual = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 2, 3, 4, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], Best fitness = 0.002459863856897593
=== Gen1 === 
Generation 1: Best individual = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 2, 3, 4, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], Best fitness = 0.002459863856897593
=== Gen2 === 
Generation 2: Best individual = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 2, 3, 4, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]], Best fitness = 0.0024853312830756635
=== Gen3 === 
Generation 3: Best individual = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 2, 3, 4, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]], Best fitness = 0.00253223812681996
=== Gen4 === 
Generation 4: Best individual = [[0, 1, 

In [227]:
def print_routes(individual):
    # Prints out the routes taken by each vehicle in the individual
    for i in range(len(individual)):
        vehicle = vehicle_list[i]
        deliveries = individual[i]
        if len(deliveries) == 0:
            print(f'Vehicle {i+1} did not make any deliveries.')
            continue
        # Print out the starting location of the vehicle
        print(f'Vehicle {i+1} starting location: ({vehicle[0][0]}, {vehicle[1][0]})')
        # Print out the delivery locations
        for j in range(len(deliveries)):
            location = location_list[deliveries[j]]
            print(f'  Delivery {j+1}: ({location[0][0]}, {location[1][0]})')
        # Print out the ending location of the vehicle
        print(f'  Vehicle {i+1} ending location: ({vehicle[0][0]}, {vehicle[1][0]})')
        print('\n')

# Call this function at the end of the main loop to print out the routes of the best individual
best_individual = sorted(population, key=lambda x: calculate_fitness(x), reverse=True)[0]
print_routes(best_individual)

Vehicle 1 did not make any deliveries.
Vehicle 2 starting location: (0, 0)
  Delivery 1: (6, 4)
  Delivery 2: (8, 3)
  Delivery 3: (6, 4)
  Delivery 4: (4, 7)
  Delivery 5: (3, 8)
  Delivery 6: (2, 5)
  Delivery 7: (4, 7)
  Vehicle 2 ending location: (0, 0)


Vehicle 3 starting location: (0, 0)
  Delivery 1: (1, 2)
  Delivery 2: (2, 5)
  Delivery 3: (5, 6)
  Delivery 4: (5, 6)
  Delivery 5: (9, 9)
  Delivery 6: (5, 6)
  Delivery 7: (1, 2)
  Delivery 8: (2, 5)
  Delivery 9: (3, 8)
  Delivery 10: (6, 4)
  Delivery 11: (5, 6)
  Vehicle 3 ending location: (0, 0)


Vehicle 4 starting location: (0, 0)
  Delivery 1: (9, 9)
  Delivery 2: (8, 3)
  Delivery 3: (1, 2)
  Delivery 4: (7, 1)
  Delivery 5: (2, 5)
  Delivery 6: (4, 7)
  Delivery 7: (4, 7)
  Delivery 8: (4, 7)
  Delivery 9: (2, 5)
  Delivery 10: (10, 10)
  Delivery 11: (9, 9)
  Delivery 12: (9, 9)
  Delivery 13: (8, 3)
  Delivery 14: (10, 10)
  Delivery 15: (3, 8)
  Delivery 16: (1, 2)
  Delivery 17: (6, 4)
  Delivery 18: (7, 1)
  Deli